In [8]:
import duckdb
import pandas as  pd

%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [6]:
%sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db

In [7]:
%%sql

SELECT * FROM duckdb_extensions();

,extension_name,loaded,installed,install_path,description,aliases
0,arrow,False,False,,A zero-copy data integration between Apache Ar...,[]
1,autocomplete,False,False,,Adds support for autocomplete in the shell,[]
2,aws,False,False,,Provides features that depend on the AWS SDK,[]
3,azure,False,False,,Adds a filesystem abstraction for Azure blob s...,[]
4,excel,False,False,,Adds support for Excel-like format strings,[]
5,fts,False,True,(BUILT-IN),Adds support for Full-Text Search Indexes,[]
6,httpfs,False,False,,Adds support for reading and writing files ove...,"[http, https, s3]"
7,iceberg,False,False,,Adds support for Apache Iceberg,[]
8,icu,True,True,(BUILT-IN),Adds support for time zones and collations usi...,[]
9,inet,False,False,,Adds support for IP-related data types and fun...,[]
